# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv('../output_data/cities.csv')
cities_df


,Unnamed: 0,city,lat,lon,max temp,humidity,cloudiness,wind speed,country,date
0,0,Mogadishu,2.0371,45.3438,77.67,81,67,23.64,SO,1644430843
1,1,George Town,5.4112,100.3354,80.53,95,20,3.44,MY,1644430895
2,2,Mahina,-17.5065,-149.4890,82.17,74,20,5.75,PF,1644430940
3,3,Miyako,39.6367,141.9525,32.52,71,100,1.70,JP,1644430941
4,4,Saint Paul Harbor,57.7900,-152.4072,24.31,63,0,8.05,US,1644430941
...,...,...,...,...,...,...,...,...,...,...
595,595,Kachīry,53.0675,76.1036,-12.50,98,7,5.79,KZ,1644431133
596,596,Koumac,-20.5667,164.2833,80.11,89,100,22.28,NC,1644431133
597,597,Fall River,41.7015,-71.1551,44.94,55,0,7.00,US,1644431134
598,598,Sirte,31.2089,16.5887,57.36,59,97,14.32,LY,1644431134


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [25]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [26]:
# Store latitude and longitude in locations
locations = cities_df[["lat", "lon"]].astype(float)
humidity = cities_df["humidity"].astype(float)

In [31]:
# Create a poverty Heatmap layer
fig = gmaps.figure(center=(13.0, -60), zoom_level=3)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# creating new DF for ideal weather conditions
ideal_weather_df = cities_df.loc[(cities_df['max temp'] >= 75) &
                        (cities_df['max temp'] <= 95) &
                        (cities_df['wind speed'] < 10) &
                        (cities_df['cloudiness'] < 50) &
                        (cities_df['humidity'] < 60)]
ideal_weather_df

,Unnamed: 0,city,lat,lon,max temp,humidity,cloudiness,wind speed,country,date
7,7,Rocha,-34.4833,-54.3333,77.92,46,7,8.52,UY,1644430942
60,60,Malibu,34.0050,-118.8101,83.68,15,0,6.46,US,1644430958
92,92,Artigas,-30.4000,-56.4667,88.90,21,0,2.55,UY,1644430969
100,100,Yulara,-25.2406,130.9889,75.04,53,0,5.75,AU,1644430971
117,117,Conakry,9.5380,-13.6773,86.07,58,29,6.91,GN,1644430976
143,143,Venado Tuerto,-33.7456,-61.9688,84.78,24,0,8.23,AR,1644430985
180,180,Departamento de Maldonado,-34.6667,-54.9167,76.39,42,4,5.48,UY,1644430995
196,196,Boa Vista,2.8197,-60.6733,94.98,34,40,6.91,BR,1644431000
207,207,Mattru,7.6244,-11.8332,87.30,55,47,7.65,SL,1644431005
212,212,Pochutla,15.7432,-96.4661,86.83,44,5,5.91,MX,1644431006


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# from 6-APIs, class 3, 05-Stu_Google_Complex
hotel_df = pd.DataFrame(ideal_weather_df, columns=['city', 'country', 'lat', 'lon'])

hotel_df['hotel name'] = ""
hotel_df

,city,country,lat,lon,hotel name
7,Rocha,UY,-34.4833,-54.3333,
60,Malibu,US,34.0050,-118.8101,
92,Artigas,UY,-30.4000,-56.4667,
100,Yulara,AU,-25.2406,130.9889,
117,Conakry,GN,9.5380,-13.6773,
143,Venado Tuerto,AR,-33.7456,-61.9688,
180,Departamento de Maldonado,UY,-34.6667,-54.9167,
196,Boa Vista,BR,2.8197,-60.6733,
207,Mattru,SL,7.6244,-11.8332,
212,Pochutla,MX,15.7432,-96.4661,


In [8]:
#copied directly from 6-APIs, class 3, 05-Stu_Google_Complex

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["lat"]
    lon = row["lon"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lon}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    print(f"retrieving results for {index}: {row['city']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))
    
    results = response['results']

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        print(f"the closest hotel is {results [0] ['name']}.")
        hotel_df.loc[index, "hotel name"] = response["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    print("--------------")

retrieving results for 7: Rocha.
the closest hotel is Hotel Arrarte.
--------------
retrieving results for 60: Malibu.
the closest hotel is Malibu Country Inn.
--------------
retrieving results for 92: Artigas.
the closest hotel is Hotel Casino San Eugenio del Cuareim.
--------------
retrieving results for 100: Yulara.
the closest hotel is Desert Gardens Hotel - Ayers Rock Resort.
--------------
retrieving results for 117: Conakry.
the closest hotel is Riviera Royal Hotel.
--------------
retrieving results for 143: Venado Tuerto.
the closest hotel is Hotel - Miro Park.
--------------
retrieving results for 180: Departamento de Maldonado.
the closest hotel is La Cimarrona.
--------------
retrieving results for 196: Boa Vista.
the closest hotel is Aipana Plaza Hotel.
--------------
retrieving results for 207: Mattru.
the closest hotel is Mondkor Town, Bagbo Chiefdom, Bo District.
--------------
retrieving results for 212: Pochutla.
the closest hotel is Hotel Posada San Jose.
------------

In [9]:
hotel_df

,city,country,lat,lon,hotel name
7,Rocha,UY,-34.4833,-54.3333,Hotel Arrarte
60,Malibu,US,34.0050,-118.8101,Malibu Country Inn
92,Artigas,UY,-30.4000,-56.4667,Hotel Casino San Eugenio del Cuareim
100,Yulara,AU,-25.2406,130.9889,Desert Gardens Hotel - Ayers Rock Resort
117,Conakry,GN,9.5380,-13.6773,Riviera Royal Hotel
143,Venado Tuerto,AR,-33.7456,-61.9688,Hotel - Miro Park
180,Departamento de Maldonado,UY,-34.6667,-54.9167,La Cimarrona
196,Boa Vista,BR,2.8197,-60.6733,Aipana Plaza Hotel
207,Mattru,SL,7.6244,-11.8332,"Mondkor Town, Bagbo Chiefdom, Bo District"
212,Pochutla,MX,15.7432,-96.4661,Hotel Posada San Jose


In [36]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lon"]]

In [37]:
hotel_layer = gmaps.symbol_layer(
            locations, fill_color='red',
            stroke_color='red', scale=2,
            info_box_content = (hotel_info))

fig = gmaps.figure(center=(13.0, -50), zoom_level=3)
fig.add_layer(hotel_layer)
#fig.add_layer(marker_layer)
fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))